In [1]:
### Pacotes necessarios:
# install.packages("e1071")
# install.packages("caret")
# install.packages("kernlab")
# install.packages("Metrics")
library("kernlab")
library("caret")
library(mlbench)
library("Metrics")

Carregando pacotes exigidos: ggplot2


Anexando pacote: 'ggplot2'


O seguinte objeto é mascarado por 'package:kernlab':

    alpha


Carregando pacotes exigidos: lattice


Anexando pacote: 'Metrics'


Os seguintes objetos são mascarados por 'package:caret':

    precision, recall




In [2]:
#carregamernto da base
dados <- read.csv("databases/2 - Volume - Dados.csv")
head(dados)

,PROJETO,CLONE,Idade,Hdom,AreaBasal,Volume
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,Fazenda Velha,C3,21.5,8.8,3.2,12.1
2,Fazenda Velha,A3,21.9,10.2,3.9,66.8
3,Fazenda Velha,A3,26.1,10.1,4.2,72.1
4,Fazenda Velha,A4,22.0,10.0,4.2,35.9
5,Parada,B7,23.4,15.3,4.3,44.5
6,Fazenda Velha,A2,25.9,10.5,4.4,63.9


In [3]:
# separação da base em 80% 20%
set.seed(2034)

indices <- createDataPartition(dados$Volume, p = 0.80, lis = FALSE)

treino <- dados[indices, ]
teste <- dados[-indices, ]

In [4]:
# fazendo o treinamento do modelo de svmRadial usando hold-out

set.seed(2034)

svm <- train(Volume ~ ., data = treino, method = "svmRadial")
svm

Support Vector Machines with Radial Basis Function Kernel 

246 samples
  5 predictor

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 246, 246, 246, 246, 246, 246, ... 
Resampling results across tuning parameters:

  C     RMSE      Rsquared   MAE     
  0.25  38.82743  0.9688585  26.80030
  0.50  29.21977  0.9806463  20.72325
  1.00  24.34270  0.9857847  17.73396

Tuning parameter 'sigma' was held constant at a value of 0.08385485
RMSE was used to select the optimal model using the smallest value.
The final values used for the model were sigma = 0.08385485 and C = 1.

In [5]:
#aplicando o modelo na base de teste

predict_svm <- predict(svm, teste)

#calculando as metricas
rmse(teste$Volume, predict_svm)

r2 <- function(predito, observado) {
  return(1 - (sum((predito - observado)^2) /
                sum((observado - mean(observado))^2)))
}

r2(predict_svm, teste$Volume)

[1] 18.85413

[1] 0.9905857

In [8]:
#usando cross-validation
ctrl <- trainControl(method = "cv", number = 10)

set.seed(2034)
svm <- train(Volume ~ ., data = treino, method = "svmRadial", trControl = ctrl)
svm

Support Vector Machines with Radial Basis Function Kernel 

246 samples
  5 predictor

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 222, 221, 219, 222, 222, 222, ... 
Resampling results across tuning parameters:

  C     RMSE      Rsquared   MAE     
  0.25  35.77839  0.9775831  25.58888
  0.50  26.56577  0.9863196  19.40650
  1.00  20.81773  0.9904685  15.60169

Tuning parameter 'sigma' was held constant at a value of 0.08385485
RMSE was used to select the optimal model using the smallest value.
The final values used for the model were sigma = 0.08385485 and C = 1.

In [9]:
#aplicando os modelos treinados na base de teste

predict_svm <- predict(svm, teste)

#calculando as metricas
rmse(teste$Volume, predict_svm)
r2(predict_svm, teste$Volume)

[1] 18.85413

[1] 0.9905857

In [10]:
# testando outros valores de C e sigma
tune_grid <- expand.grid(C = c(1, 2, 10, 50, 100), sigma = c(.021, .015, 0.2))

set.seed(2034)
svm <- train(Volume ~ ., data = treino, method = "svmRadial", trControl = ctrl, tuneGrid = tune_grid)
svm

Support Vector Machines with Radial Basis Function Kernel 

246 samples
  5 predictor

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 222, 221, 219, 222, 222, 222, ... 
Resampling results across tuning parameters:

  C    sigma  RMSE      Rsquared   MAE     
    1  0.015  21.80925  0.9901580  15.95344
    1  0.021  19.21112  0.9922378  14.61384
    1  0.200  30.88650  0.9811579  21.04720
    2  0.015  16.44322  0.9938854  12.67541
    2  0.021  15.57157  0.9946196  12.27378
    2  0.200  26.69831  0.9846475  18.24542
   10  0.015  12.87720  0.9961190  10.50150
   10  0.021  13.91277  0.9958098  11.69571
   10  0.200  25.80991  0.9855112  17.88841
   50  0.015  12.68891  0.9962101  10.46622
   50  0.021  13.91510  0.9958090  11.69488
   50  0.200  25.80991  0.9855112  17.88841
  100  0.015  12.68891  0.9962101  10.46622
  100  0.021  13.91510  0.9958090  11.69488
  100  0.200  25.80991  0.9855112  17.88841

RMSE was used to select the optimal model usi

In [11]:
#aplicando o modelo na base de teste
predict_svm <- predict(svm, teste)

#calculando as metricas
rmse(teste$Volume, predict_svm)
r2(predict_svm, teste$Volume)

[1] 10.40387

[1] 0.9971334

In [12]:
#predicoes de novos casos
dados_novos_casos <- read.csv("databases/2 - Volume - Dados - Novos Casos.csv")
View(dados_novos_casos)

dados_novos_casos$Volume <- NULL
predict_svm <- predict(svm, dados_novos_casos)
resultado <- cbind(dados_novos_casos, predict_svm)
View(resultado)

PROJETO,CLONE,Idade,Hdom,AreaBasal,Volume
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
Parada,C3,21.5,8.8,4.2,?
Santa Maria,A3,22.9,14.1,5.6,?
Jacutinga,A2,59.4,22.3,13.0,?


PROJETO,CLONE,Idade,Hdom,AreaBasal,predict_svm
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Parada,C3,21.5,8.8,4.2,23.45296
Santa Maria,A3,22.9,14.1,5.6,142.16872
Jacutinga,A2,59.4,22.3,13.0,294.02829
